# 前処理とベースラインモデル構築
## Kaggle House Prices: Advanced Regression Techniques

このノートブックでは、Kaggle住宅価格予測コンペティションのデータに対して、以下のステップを実行します。

1. **データの読み込み** - 訓練データとテストデータの読み込み、目的変数の対数変換
2. **外れ値の除去** - EDAで発見した異常値を除去
3. **欠損値の処理** - 欠損値の意味を理解し、適切に補完
4. **特徴量エンジニアリング** - ドメイン知識を活用して新しい特徴量を作成
5. **エンコーディング** - カテゴリカル変数を数値に変換
6. **スケーリング** - 特徴量のスケールを統一
7. **ベースラインモデル（Ridge回帰）** - 最初のモデルを構築
8. **モデル比較** - 複数のモデルを比較して最適なものを選択
9. **提出ファイル作成** - Kaggleに提出する予測ファイルを生成

**学習目標**: 各ステップの「なぜ」を理解し、機械学習パイプラインの全体像を把握する

---
## 0. ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

%matplotlib inline

print('ライブラリのインポート完了')

---
## 1. データの読み込み

### なぜ目的変数を対数変換するのか？

住宅価格（`SalePrice`）は**右に歪んだ分布**（正の歪度）を持っています。つまり、安い家が多く、高い家は少ないという分布です。

線形回帰をはじめとする多くのモデルは、目的変数が**正規分布に近い**ことを仮定しています。対数変換を行うことで：

- 分布が正規分布に近づく
- 極端に高い価格のデータの影響が緩和される
- モデルの予測精度が向上する
- Kaggleのこのコンペティションの評価指標がRMSLE（Root Mean Squared Log Error）であるため、対数変換した値に対するRMSEが評価指標と一致する

`np.log1p(x)` は `np.log(1 + x)` と同じです。`1` を足すのは、値が `0` の場合に `log(0)` が `-inf` になるのを防ぐためです。

### なぜ訓練データとテストデータを結合するのか？

前処理（欠損値補完、エンコーディング等）を訓練データとテストデータで**一貫して**行うためです。別々に処理すると、ダミー変数のカラム数が異なるなどの問題が発生します。

In [ ]:
# データの読み込み
train = pd.read_csv('/home/rex/Documents/lb/Kaggle/house-price-prediction/data/raw/train.csv')
test = pd.read_csv('/home/rex/Documents/lb/Kaggle/house-price-prediction/data/raw/test.csv')

print(f'訓練データ: {train.shape[0]}行 × {train.shape[1]}列')
print(f'テストデータ: {test.shape[0]}行 × {test.shape[1]}列')

In [ ]:
# 目的変数の分布を確認（対数変換前後の比較）
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 変換前
axes[0].hist(train['SalePrice'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('SalePrice（変換前）', fontsize=14)
axes[0].set_xlabel('SalePrice')
axes[0].set_ylabel('頻度')
skew_before = train['SalePrice'].skew()
axes[0].text(0.65, 0.85, f'歪度: {skew_before:.3f}', transform=axes[0].transAxes, fontsize=12,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 変換後
axes[1].hist(np.log1p(train['SalePrice']), bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_title('log1p(SalePrice)（変換後）', fontsize=14)
axes[1].set_xlabel('log1p(SalePrice)')
axes[1].set_ylabel('頻度')
skew_after = np.log1p(train['SalePrice']).skew()
axes[1].text(0.65, 0.85, f'歪度: {skew_after:.3f}', transform=axes[1].transAxes, fontsize=12,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f'\n対数変換により歪度が {skew_before:.3f} → {skew_after:.3f} に改善されました')
print('歪度が0に近いほど正規分布に近い形状です')

In [ ]:
# 目的変数を分離して対数変換
y_train = np.log1p(train['SalePrice'])

# Idを保存（後で提出ファイルに使用）
train_id = train['Id']
test_id = test['Id']

# 訓練データの行数を記録（後で分割に使用）
n_train = train.shape[0]

# SalePriceとIdを除いて結合
train.drop(['SalePrice', 'Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

# 訓練データとテストデータを結合
all_data = pd.concat([train, test], axis=0, ignore_index=True)

print(f'結合データ: {all_data.shape[0]}行 × {all_data.shape[1]}列')
print(f'  - 訓練データ: {n_train}行（インデックス 0 〜 {n_train-1}）')
print(f'  - テストデータ: {all_data.shape[0] - n_train}行（インデックス {n_train} 〜 {all_data.shape[0]-1}）')

---
## 2. 外れ値の除去

### なぜ外れ値を除去するのか？

EDA（探索的データ分析）で、`GrLivArea`（地上階の面積）が **4000平方フィート以上** なのに `SalePrice` が非常に低い住宅が2件見つかっています。これは明らかに異常な取引（例：親族間の格安売却など）であり、モデルの学習に悪影響を与えます。

**注意**: 外れ値の除去は訓練データに対してのみ行います。テストデータの外れ値は除去できません（予測しなければならないため）。

**重要**: 外れ値の除去は慎重に行う必要があります。データの分布を理解した上で、ドメイン知識に基づいて判断します。むやみに除去すると、モデルが一般化できなくなる可能性があります。

In [ ]:
# 外れ値を可視化
fig, ax = plt.subplots(figsize=(10, 6))

# 訓練データ部分のみプロット
train_part = all_data.iloc[:n_train]
ax.scatter(train_part['GrLivArea'], y_train, alpha=0.5, color='steelblue', s=20)

# 外れ値をハイライト
outlier_mask = train_part['GrLivArea'] > 4000
ax.scatter(train_part.loc[outlier_mask, 'GrLivArea'], y_train[outlier_mask],
           color='red', s=100, marker='x', linewidths=3, label='外れ値 (GrLivArea > 4000)')

ax.set_xlabel('GrLivArea（地上階面積）', fontsize=12)
ax.set_ylabel('log1p(SalePrice)', fontsize=12)
ax.set_title('GrLivArea vs SalePrice - 外れ値の特定', fontsize=14)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f'外れ値の数: {outlier_mask.sum()}件')
print(f'外れ値のインデックス: {list(train_part[outlier_mask].index)}')

In [ ]:
# 外れ値を除去
# 訓練データ部分でGrLivArea > 4000のインデックスを特定
outlier_indices = all_data.iloc[:n_train][all_data.iloc[:n_train]['GrLivArea'] > 4000].index

# all_dataとy_trainから外れ値を除去
all_data = all_data.drop(outlier_indices)
y_train = y_train.drop(outlier_indices)

# インデックスをリセット
all_data = all_data.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# 訓練データの行数を更新
n_train = n_train - len(outlier_indices)

print(f'外れ値 {len(outlier_indices)}件を除去しました')
print(f'更新後の訓練データ: {n_train}行')
print(f'更新後の結合データ: {all_data.shape[0]}行')

---
## 3. 欠損値の処理

### 欠損値処理が重要な理由

多くの機械学習アルゴリズムは欠損値（NaN）を扱えません。そのため、学習前に欠損値を適切に処理する必要があります。

### 欠損値の種類と処理方針

この住宅データでは、欠損値には**2つの意味**があります：

1. **「その設備がない」ことを意味するNA**: 例えば `PoolQC`（プール品質）がNAの場合、「プールがない」ことを意味します。この場合、カテゴリカル変数は `"None"`、数値変数は `0` で埋めるのが適切です。

2. **本当にデータが不明なNA**: 例えば `LotFrontage`（道路接面距離）のNAは、データが記録されていないだけです。この場合、統計量（中央値やモード）で補完します。

### なぜ中央値（median）を使うのか？

平均値は外れ値の影響を受けやすいですが、中央値は**ロバスト**（外れ値に強い）です。住宅データには極端な値が含まれることが多いため、中央値が安全な選択です。

### LotFrontageをNeighborhoodごとの中央値で埋める理由

同じ地域（Neighborhood）の住宅は似たような区画サイズを持つ傾向があります。全体の中央値よりも、地域ごとの中央値の方が正確な推定値になります。

In [ ]:
# 欠損値の状況を確認
missing = all_data.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

print(f'欠損値を持つ特徴量の数: {len(missing)}')
print('\n--- 欠損値の一覧（処理前） ---')
missing_df = pd.DataFrame({
    '欠損数': missing,
    '欠損率(%)': (missing / len(all_data) * 100).round(2)
})
print(missing_df.to_string())

In [ ]:
# 欠損値の可視化
fig, ax = plt.subplots(figsize=(12, 8))
missing_pct = (missing / len(all_data) * 100)
colors = ['#d32f2f' if x > 50 else '#ff9800' if x > 10 else '#4caf50' for x in missing_pct.values]
bars = ax.barh(range(len(missing_pct)), missing_pct.values, color=colors, edgecolor='black', alpha=0.8)
ax.set_yticks(range(len(missing_pct)))
ax.set_yticklabels(missing_pct.index, fontsize=9)
ax.set_xlabel('欠損率 (%)', fontsize=12)
ax.set_title('特徴量ごとの欠損率（処理前）', fontsize=14)
ax.invert_yaxis()

# 凡例
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#d32f2f', edgecolor='black', label='50%超'),
    Patch(facecolor='#ff9800', edgecolor='black', label='10-50%'),
    Patch(facecolor='#4caf50', edgecolor='black', label='10%未満')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# ステップ1: NAが「その設備がない」を意味する特徴量
# ============================================================

# --- カテゴリカル変数: "None" で埋める ---
# これらの特徴量では、NAは「その設備/機能が存在しない」ことを意味する
none_cols_cat = [
    'PoolQC',        # プール品質 → プールなし
    'MiscFeature',   # その他設備 → 特殊設備なし
    'Alley',         # 路地のタイプ → 路地なし
    'Fence',         # フェンス品質 → フェンスなし
    'FireplaceQu',   # 暖炉品質 → 暖炉なし
    'GarageType',    # ガレージタイプ → ガレージなし
    'GarageFinish',  # ガレージ内装 → ガレージなし
    'GarageQual',    # ガレージ品質 → ガレージなし
    'GarageCond',    # ガレージ状態 → ガレージなし
    'BsmtQual',      # 地下室品質 → 地下室なし
    'BsmtCond',      # 地下室状態 → 地下室なし
    'BsmtExposure',  # 地下室露出 → 地下室なし
    'BsmtFinType1',  # 地下室仕上げ1 → 地下室なし
    'BsmtFinType2',  # 地下室仕上げ2 → 地下室なし
    'MasVnrType',    # 石材外装タイプ → 石材外装なし
]

for col in none_cols_cat:
    all_data[col] = all_data[col].fillna('None')
    
print(f'カテゴリカル変数 {len(none_cols_cat)}列を "None" で補完しました')

# --- 数値変数: 0 で埋める ---
# ガレージや地下室がない場合、関連する数値は0
none_cols_num = [
    'GarageYrBlt',   # ガレージ建築年 → ガレージなし
    'GarageArea',    # ガレージ面積 → 0
    'GarageCars',    # ガレージ車台数 → 0
    'BsmtFinSF1',    # 地下室仕上げ面積1 → 0
    'BsmtFinSF2',    # 地下室仕上げ面積2 → 0
    'BsmtUnfSF',     # 地下室未仕上げ面積 → 0
    'TotalBsmtSF',   # 地下室総面積 → 0
    'BsmtFullBath',  # 地下室フルバス → 0
    'BsmtHalfBath',  # 地下室ハーフバス → 0
    'MasVnrArea',    # 石材外装面積 → 0
]

for col in none_cols_num:
    all_data[col] = all_data[col].fillna(0)

print(f'数値変数 {len(none_cols_num)}列を 0 で補完しました')

In [ ]:
# ============================================================
# ステップ2: LotFrontage - Neighborhoodごとの中央値で補完
# ============================================================

# 各Neighborhoodの中央値を計算
lot_frontage_by_neighborhood = all_data.groupby('Neighborhood')['LotFrontage'].median()

print('Neighborhoodごとの LotFrontage 中央値（一部表示）:')
print(lot_frontage_by_neighborhood.head(10))

# Neighborhoodごとの中央値で補完
missing_lotfrontage = all_data['LotFrontage'].isnull().sum()
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)

print(f'\nLotFrontage: {missing_lotfrontage}件の欠損値をNeighborhoodごとの中央値で補完しました')

In [ ]:
# ============================================================
# ステップ3: 残りの欠損値を処理
# ============================================================

# 残りの欠損値を確認
remaining_missing = all_data.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
print('残りの欠損値:')
print(remaining_missing)
print()

In [ ]:
# カテゴリカル変数: 最頻値（モード）で補完
# 理由: カテゴリカル変数の「平均」は意味がないため、最も多いカテゴリで埋める
remaining_cat = all_data.select_dtypes(include='object').columns
cat_filled_count = 0
for col in remaining_cat:
    if all_data[col].isnull().sum() > 0:
        mode_val = all_data[col].mode()[0]
        n_missing = all_data[col].isnull().sum()
        all_data[col] = all_data[col].fillna(mode_val)
        cat_filled_count += 1
        print(f'  {col}: {n_missing}件 → 最頻値 "{mode_val}" で補完')

print(f'\nカテゴリカル変数 {cat_filled_count}列を最頻値で補完しました')

# 数値変数: 中央値で補完
remaining_num = all_data.select_dtypes(include=[np.number]).columns
num_filled_count = 0
for col in remaining_num:
    if all_data[col].isnull().sum() > 0:
        median_val = all_data[col].median()
        n_missing = all_data[col].isnull().sum()
        all_data[col] = all_data[col].fillna(median_val)
        num_filled_count += 1
        print(f'  {col}: {n_missing}件 → 中央値 {median_val} で補完')

print(f'\n数値変数 {num_filled_count}列を中央値で補完しました')

In [ ]:
# 最終確認: 欠損値が残っていないことを確認
total_missing = all_data.isnull().sum().sum()
print(f'\n=== 欠損値処理完了 ===')
print(f'残りの欠損値の総数: {total_missing}')

if total_missing == 0:
    print('全ての欠損値が正常に処理されました！')
else:
    print('警告: まだ欠損値が残っています')
    print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

---
## 4. 特徴量エンジニアリング

### 特徴量エンジニアリングとは？

既存のデータから**新しい特徴量**を作成することで、モデルが住宅価格をより正確に予測できるようにします。ドメイン知識（住宅市場の知識）を活用して、価格に影響する要因を明示的に表現します。

### なぜ新しい特徴量を作るのか？

機械学習モデルは、与えられた特徴量から**パターン**を学習します。重要な情報が明示的に特徴量として存在する方が、モデルはより効果的に学習できます。

例えば「総面積」は1階・2階・地下室の面積の合計ですが、モデルがこの合計の概念を自動的に学習するのは難しい（特に線形モデルの場合）ため、明示的に作成します。

In [ ]:
# ============================================================
# 新しい特徴量の作成
# ============================================================

# --- TotalSF: 総面積 ---
# 直感: 住宅の総面積は価格の最も重要な決定要因の一つ。
# 地下室 + 1階 + 2階を合計することで、住宅全体の広さを表現。
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
print('TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF')
print(f'  範囲: {all_data["TotalSF"].min():.0f} 〜 {all_data["TotalSF"].max():.0f}')

# --- TotalBathrooms: バスルーム合計 ---
# 直感: バスルーム数は住宅の快適性を表す。
# ハーフバス（洗面台+トイレのみ）はフルバスの半分の価値として計算。
all_data['TotalBathrooms'] = (
    all_data['FullBath'] + 
    0.5 * all_data['HalfBath'] + 
    all_data['BsmtFullBath'] + 
    0.5 * all_data['BsmtHalfBath']
)
print('\nTotalBathrooms = FullBath + 0.5*HalfBath + BsmtFullBath + 0.5*BsmtHalfBath')
print(f'  範囲: {all_data["TotalBathrooms"].min():.1f} 〜 {all_data["TotalBathrooms"].max():.1f}')

# --- TotalPorchSF: ポーチ総面積 ---
# 直感: ポーチ（玄関先の屋根付きスペース）の総面積。
# アウトドアリビングスペースとして住宅の魅力を高める。
all_data['TotalPorchSF'] = (
    all_data['OpenPorchSF'] + 
    all_data['EnclosedPorch'] + 
    all_data['3SsnPorch'] + 
    all_data['ScreenPorch']
)
print('\nTotalPorchSF = OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch')
print(f'  範囲: {all_data["TotalPorchSF"].min():.0f} 〜 {all_data["TotalPorchSF"].max():.0f}')

# --- バイナリフラグ（有/無の特徴量） ---
# 直感: 設備の「有無」自体が価格に大きく影響することがある。
# 例: プールがある家は少数派であり、その存在自体が価値を持つ。
all_data['HasPool'] = (all_data['PoolArea'] > 0).astype(int)
all_data['HasGarage'] = (all_data['GarageArea'] > 0).astype(int)
all_data['HasFireplace'] = (all_data['Fireplaces'] > 0).astype(int)
print('\nバイナリフラグ:')
print(f'  HasPool: プールあり {all_data["HasPool"].sum()}件 / {len(all_data)}件')
print(f'  HasGarage: ガレージあり {all_data["HasGarage"].sum()}件 / {len(all_data)}件')
print(f'  HasFireplace: 暖炉あり {all_data["HasFireplace"].sum()}件 / {len(all_data)}件')

# --- HouseAge: 築年数 ---
# 直感: 新しい家ほど高い傾向がある。YearBuiltよりも「何年前に建てたか」の方が直感的。
all_data['HouseAge'] = all_data['YrSold'] - all_data['YearBuilt']
print(f'\nHouseAge = YrSold - YearBuilt')
print(f'  範囲: {all_data["HouseAge"].min()} 〜 {all_data["HouseAge"].max()}年')

# --- RemodAge: リフォームからの年数 ---
# 直感: 最近リフォームした家は状態が良く、高く売れる傾向がある。
all_data['RemodAge'] = all_data['YrSold'] - all_data['YearRemodAdd']
print(f'\nRemodAge = YrSold - YearRemodAdd')
print(f'  範囲: {all_data["RemodAge"].min()} 〜 {all_data["RemodAge"].max()}年')

In [ ]:
# 新しい特徴量と目的変数の相関を確認（訓練データのみ）
new_features = ['TotalSF', 'TotalBathrooms', 'TotalPorchSF', 'HasPool', 
                'HasGarage', 'HasFireplace', 'HouseAge', 'RemodAge']

fig, axes = plt.subplots(2, 4, figsize=(18, 8))
axes = axes.flatten()

for i, feat in enumerate(new_features):
    ax = axes[i]
    train_feat = all_data.iloc[:n_train][feat]
    corr = np.corrcoef(train_feat, y_train)[0, 1]
    ax.scatter(train_feat, y_train, alpha=0.3, s=10, color='steelblue')
    ax.set_xlabel(feat, fontsize=10)
    ax.set_ylabel('log1p(SalePrice)', fontsize=9)
    ax.set_title(f'{feat}\n(相関: {corr:.3f})', fontsize=11)

plt.suptitle('新しい特徴量と目的変数の関係', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print('\n各新特徴量とSalePriceの相関係数:')
for feat in new_features:
    corr = np.corrcoef(all_data.iloc[:n_train][feat], y_train)[0, 1]
    print(f'  {feat:20s}: {corr:+.4f}')

---
## 5. エンコーディング

### なぜエンコーディングが必要なのか？

機械学習モデルは**数値データ**しか扱えません。カテゴリカル変数（文字列）を数値に変換する必要があります。

### 順序変数（Ordinal）と名義変数（Nominal）の違い

- **順序変数（Ordinal）**: カテゴリに**自然な順序**がある変数。例: 品質（Excellent > Good > Average > Fair > Poor）。ラベルエンコーディング（数値を割り当て）が適切。

- **名義変数（Nominal）**: カテゴリに**順序がない**変数。例: 屋根の材質（Gable, Hip, Flat, ...）。ワンホットエンコーディング（各カテゴリを0/1の列に展開）が適切。

### なぜこの区別が重要か？

名義変数にラベルエンコーディングを使うと、モデルが「数値の大小関係」を誤って学習してしまいます。例えば、地域を A=1, B=2, C=3 とエンコードすると、モデルは「CはAの3倍」のような誤った関係を学習する可能性があります。

### `drop_first=True` の理由

ワンホットエンコーディングで `drop_first=True` にすると、最初のカテゴリの列を削除します。これは**多重共線性**（特徴量間の完全な相関）を防ぐためです。例えば、2つのカテゴリ（A, B）がある場合、Aの列が0であればBは必ず1なので、片方の情報があれば十分です。

In [ ]:
# ============================================================
# 順序変数（Ordinal）: ラベルエンコーディング
# ============================================================

# 品質を表す順序変数のマッピング
# Ex(Excellent)=5, Gd(Good)=4, TA(Typical/Average)=3, Fa(Fair)=2, Po(Poor)=1, None=0
quality_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0}

# 品質系の順序変数
quality_cols = [
    'ExterQual',     # 外装品質
    'ExterCond',     # 外装状態
    'BsmtQual',      # 地下室品質
    'BsmtCond',      # 地下室状態
    'HeatingQC',     # 暖房品質
    'KitchenQual',   # キッチン品質
    'FireplaceQu',   # 暖炉品質
    'GarageQual',    # ガレージ品質
    'GarageCond',    # ガレージ状態
    'PoolQC',        # プール品質
]

for col in quality_cols:
    all_data[col] = all_data[col].map(quality_mapping)
    # マッピングに含まれない値がある場合は0で埋める
    all_data[col] = all_data[col].fillna(0)

print('品質系の順序変数をエンコーディングしました:')
print(f'  マッピング: {quality_mapping}')
print(f'  対象列: {quality_cols}')

# BsmtExposure: 地下室の露出度
bsmt_exposure_mapping = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0}
all_data['BsmtExposure'] = all_data['BsmtExposure'].map(bsmt_exposure_mapping).fillna(0)
print(f'\nBsmtExposure: {bsmt_exposure_mapping}')

# BsmtFinType1, BsmtFinType2: 地下室仕上げタイプ
bsmt_fin_mapping = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None': 0}
for col in ['BsmtFinType1', 'BsmtFinType2']:
    all_data[col] = all_data[col].map(bsmt_fin_mapping).fillna(0)
print(f'BsmtFinType: {bsmt_fin_mapping}')

# GarageFinish: ガレージ内装
garage_fin_mapping = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'None': 0}
all_data['GarageFinish'] = all_data['GarageFinish'].map(garage_fin_mapping).fillna(0)
print(f'GarageFinish: {garage_fin_mapping}')

# Fence: フェンス品質
fence_mapping = {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'None': 0}
all_data['Fence'] = all_data['Fence'].map(fence_mapping).fillna(0)
print(f'Fence: {fence_mapping}')

# Functional: 機能性評価
functional_mapping = {'Typ': 7, 'Min1': 6, 'Min2': 5, 'Mod': 4, 'Maj1': 3, 'Maj2': 2, 'Sev': 1, 'Sal': 0}
all_data['Functional'] = all_data['Functional'].map(functional_mapping).fillna(7)
print(f'Functional: {functional_mapping}')

# LandSlope: 土地の傾斜
land_slope_mapping = {'Gtl': 2, 'Mod': 1, 'Sev': 0}
all_data['LandSlope'] = all_data['LandSlope'].map(land_slope_mapping).fillna(2)
print(f'LandSlope: {land_slope_mapping}')

# PavedDrive: 舗装されたドライブウェイ
paved_mapping = {'Y': 2, 'P': 1, 'N': 0}
all_data['PavedDrive'] = all_data['PavedDrive'].map(paved_mapping).fillna(0)
print(f'PavedDrive: {paved_mapping}')

# CentralAir: セントラル空調
all_data['CentralAir'] = all_data['CentralAir'].map({'Y': 1, 'N': 0}).fillna(0)
print(f'CentralAir: Y=1, N=0')

# Street: 道路のタイプ
all_data['Street'] = all_data['Street'].map({'Pave': 1, 'Grvl': 0}).fillna(0)
print(f'Street: Pave=1, Grvl=0')

In [ ]:
# ============================================================
# 名義変数（Nominal）: ワンホットエンコーディング
# ============================================================

# 残りのカテゴリカル変数（object型）を確認
remaining_cat_cols = all_data.select_dtypes(include='object').columns.tolist()
print(f'ワンホットエンコーディング対象の名義変数: {len(remaining_cat_cols)}列')
print(f'列名: {remaining_cat_cols}')

# エンコーディング前のデータサイズ
print(f'\nエンコーディング前: {all_data.shape[1]}列')

# ワンホットエンコーディング
# drop_first=True: 多重共線性を防ぐために最初のカテゴリを削除
all_data = pd.get_dummies(all_data, columns=remaining_cat_cols, drop_first=True)

print(f'エンコーディング後: {all_data.shape[1]}列')
print(f'\n{all_data.shape[1] - len(remaining_cat_cols)}列の数値特徴量に展開されました')

In [ ]:
# データの型を確認（全て数値になっているか）
non_numeric = all_data.select_dtypes(exclude=[np.number]).columns.tolist()
if len(non_numeric) == 0:
    print('全ての特徴量が数値型に変換されました')
else:
    print(f'警告: 以下の列がまだ数値型ではありません: {non_numeric}')

print(f'\nデータの最終形状: {all_data.shape}')
print(f'  訓練データ: {n_train}行')
print(f'  テストデータ: {all_data.shape[0] - n_train}行')
print(f'  特徴量数: {all_data.shape[1]}列')

---
## 6. スケーリング

### なぜスケーリングが必要なのか？

特徴量によってスケール（値の範囲）が大きく異なります。例えば：
- `LotArea`（敷地面積）: 1,300 〜 200,000+
- `OverallQual`（品質評価）: 1 〜 10
- `HasPool`（プール有無）: 0 〜 1

**線形モデル**（Ridge, Lasso, ElasticNet）は、特徴量のスケールに**敏感**です。大きなスケールの特徴量がモデルを支配し、小さなスケールの特徴量が無視される可能性があります。

### StandardScalerの仕組み

各特徴量について、以下の変換を行います：

$$z = \frac{x - \mu}{\sigma}$$

- $\mu$: 平均値
- $\sigma$: 標準偏差

結果として、全ての特徴量が**平均0、標準偏差1**に標準化されます。

### 注意: ツリーベースのモデルではスケーリング不要

RandomForest や GradientBoosting などのツリーベースモデルは、特徴量の分割点を探すため、スケールに影響されません。ただし、今回は線形モデルも使用するため、スケーリングを行います。

In [ ]:
# スケーリング前の値の範囲を確認（一部の特徴量）
example_cols = ['LotArea', 'OverallQual', 'TotalSF', 'GarageArea', 'HouseAge']
existing_cols = [c for c in example_cols if c in all_data.columns]

print('スケーリング前の値の範囲:')
for col in existing_cols:
    print(f'  {col:15s}: min={all_data[col].min():10.1f}, max={all_data[col].max():10.1f}, '
          f'mean={all_data[col].mean():10.1f}, std={all_data[col].std():10.1f}')

In [ ]:
# StandardScalerの適用
scaler = StandardScaler()

# 全ての数値特徴量にスケーリングを適用
all_data_scaled = pd.DataFrame(
    scaler.fit_transform(all_data),
    columns=all_data.columns,
    index=all_data.index
)

print('StandardScalerを適用しました')
print('\nスケーリング後の値の範囲:')
for col in existing_cols:
    print(f'  {col:15s}: min={all_data_scaled[col].min():8.3f}, max={all_data_scaled[col].max():8.3f}, '
          f'mean={all_data_scaled[col].mean():8.3f}, std={all_data_scaled[col].std():8.3f}')

print('\n全ての特徴量が平均≈0、標準偏差≈1に標準化されました')

In [ ]:
# 訓練データとテストデータに再分割
X_train = all_data_scaled.iloc[:n_train].values
X_test = all_data_scaled.iloc[n_train:].values

print(f'訓練データ: X_train={X_train.shape}, y_train={y_train.shape}')
print(f'テストデータ: X_test={X_test.shape}')

---
## 7. ベースラインモデル（Ridge回帰）

### Ridge回帰とは？

Ridge回帰は、通常の線形回帰に**L2正則化**（ペナルティ項）を追加したモデルです。

通常の線形回帰の損失関数：
$$L = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Ridge回帰の損失関数：
$$L = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \alpha \sum_{j=1}^{p} w_j^2$$

**$\alpha \sum w_j^2$** がペナルティ項で、これにより：
- 係数 $w_j$ が大きくなりすぎるのを防ぐ（**過学習を抑制**）
- 多重共線性がある場合でも安定した推定ができる
- $\alpha$ は正則化の強さを制御するハイパーパラメータ

### なぜRidge回帰がベースラインに適しているか？

1. **解釈しやすい**: 線形モデルなので、各特徴量の影響が明確
2. **多重共線性に強い**: 住宅データには相関の高い特徴量が多い
3. **計算が速い**: 大量の特徴量でも素早く学習できる
4. **ベースラインとして最適**: より複雑なモデルの性能比較の基準になる

### 交差検証（Cross-Validation）とは？

データを K 個の部分に分割し、各部分を順番にテストデータとして使い、残りで学習します。これにより、モデルの性能をより**信頼性高く**評価できます。

1つの分割だけでは「たまたまテストデータが簡単/難しかった」可能性がありますが、K回繰り返すことで平均的な性能がわかります。

In [ ]:
# ============================================================
# 交差検証の設定
# ============================================================

# 5分割交差検証
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# RMSE を計算する関数
def rmse_cv(model, X, y):
    """
    交差検証でRMSEを計算する関数。
    
    sklearnのcross_val_scoreは「大きいほど良い」スコアを返すため、
    neg_mean_squared_error（負のMSE）を使い、符号を反転してからルートを取る。
    """
    scores = cross_val_score(
        model, X, y, 
        scoring='neg_mean_squared_error',
        cv=kf
    )
    rmse_scores = np.sqrt(-scores)
    return rmse_scores

print('交差検証の設定:')
print('  - 分割数: 5')
print('  - シャッフル: あり（random_state=42）')
print('  - 評価指標: RMSE（Root Mean Squared Error）')
print('\n注意: 目的変数はlog1p変換済みなので、このRMSEは実質的にRMSLE相当です')

In [ ]:
# ============================================================
# Ridge回帰モデルの訓練と評価
# ============================================================

# Ridge回帰（alpha=10はよく使われるデフォルト値）
ridge = Ridge(alpha=10, random_state=42)

# 交差検証
ridge_scores = rmse_cv(ridge, X_train, y_train)

print('=== Ridge回帰（ベースライン）の結果 ===')
print(f'\n各フォールドのRMSE:')
for i, score in enumerate(ridge_scores):
    print(f'  Fold {i+1}: {score:.5f}')
print(f'\n平均RMSE: {ridge_scores.mean():.5f} (+/- {ridge_scores.std():.5f})')
print(f'\nこの値はlog1p(SalePrice)空間でのRMSEです')
print(f'参考: Kaggleのリーダーボードスコア（RMSLE）と近い値になります')

---
## 8. モデル比較

### 各モデルの概要

| モデル | 特徴 | 正則化 |
|--------|------|--------|
| **Ridge** | L2正則化。全ての特徴量を使い、係数を小さくする | $\alpha \sum w^2$ |
| **Lasso** | L1正則化。不要な特徴量の係数を完全に0にする（特徴量選択効果） | $\alpha \sum |w|$ |
| **ElasticNet** | L1とL2の組み合わせ。Lassoの特徴量選択とRidgeの安定性を両立 | $\alpha_1 \sum |w| + \alpha_2 \sum w^2$ |
| **RandomForest** | 多数の決定木の多数決。過学習に強い。特徴量のスケールに依存しない | - |
| **GradientBoosting** | 決定木を逐次的に追加し、前のモデルの誤差を修正していく | - |
| **XGBoost** | GradientBoostingの高速・高精度版。正則化機能を内蔵 | - |

### 線形モデル vs ツリーベースモデル

- **線形モデル**（Ridge, Lasso, ElasticNet）: 特徴量と目的変数の線形関係を仮定。解釈しやすいが、非線形な関係を捉えにくい。
- **ツリーベースモデル**（RandomForest, GradientBoosting, XGBoost）: 非線形な関係も捉えられる。一般的に予測精度が高い。

In [ ]:
# ============================================================
# 複数モデルの比較
# ============================================================

# モデルの定義
models = {
    'Ridge': Ridge(alpha=10, random_state=42),
    'Lasso': Lasso(alpha=0.0005, random_state=42),
    'ElasticNet': ElasticNet(alpha=0.0005, l1_ratio=0.5, random_state=42),
    'RandomForest': RandomForestRegressor(
        n_estimators=300, max_depth=15, min_samples_split=5,
        min_samples_leaf=2, random_state=42, n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=300, max_depth=4, learning_rate=0.05,
        min_samples_split=5, min_samples_leaf=3, random_state=42
    ),
}

# XGBoostを試みる（インストールされていない場合はスキップ）
try:
    from xgboost import XGBRegressor
    models['XGBoost'] = XGBRegressor(
        n_estimators=300, max_depth=4, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, n_jobs=-1, verbosity=0
    )
    print('XGBoostが利用可能です')
except ImportError:
    print('XGBoostがインストールされていません。スキップします。')
    print('インストールするには: pip install xgboost')

print(f'\n比較するモデル数: {len(models)}')
print(f'モデル一覧: {", ".join(models.keys())}')

In [ ]:
# 各モデルの交差検証を実行
results = {}

print('=== モデル比較（5分割交差検証） ===')
print(f'{"モデル":<20s} {"平均RMSE":>10s} {"標準偏差":>10s} {"最良":>10s} {"最悪":>10s}')
print('-' * 65)

for name, model in models.items():
    scores = rmse_cv(model, X_train, y_train)
    results[name] = scores
    print(f'{name:<20s} {scores.mean():>10.5f} {scores.std():>10.5f} {scores.min():>10.5f} {scores.max():>10.5f}')

# 最良モデルを特定
best_model_name = min(results, key=lambda x: results[x].mean())
best_rmse = results[best_model_name].mean()
print(f'\n最良モデル: {best_model_name} (RMSE = {best_rmse:.5f})')

In [ ]:
# 結果の可視化
fig, ax = plt.subplots(figsize=(12, 6))

model_names = list(results.keys())
means = [results[name].mean() for name in model_names]
stds = [results[name].std() for name in model_names]

# 最良モデルの色を変える
colors = ['#2196F3' if name != best_model_name else '#FF5722' for name in model_names]

bars = ax.bar(model_names, means, yerr=stds, capsize=5, color=colors,
              edgecolor='black', alpha=0.8, linewidth=1.2)

# 値をバーの上に表示
for bar, mean, std in zip(bars, means, stds):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + std + 0.002,
            f'{mean:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_ylabel('RMSE (log1p空間)', fontsize=12)
ax.set_title('モデル比較 - 5分割交差検証のRMSE\n（低いほど良い。エラーバーは標準偏差）', fontsize=14)
ax.set_ylim(bottom=min(means) * 0.9)

# 最良モデルに注釈
ax.annotate(f'Best: {best_model_name}', 
            xy=(model_names.index(best_model_name), min(means)),
            xytext=(model_names.index(best_model_name), min(means) * 0.95),
            fontsize=11, fontweight='bold', color='#FF5722',
            ha='center')

plt.xticks(rotation=15, fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# 各モデルのフォールドごとのスコアも可視化（箱ひげ図）
fig, ax = plt.subplots(figsize=(12, 6))

scores_list = [results[name] for name in model_names]
bp = ax.boxplot(scores_list, labels=model_names, patch_artist=True,
                medianprops=dict(color='black', linewidth=2))

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('RMSE (log1p空間)', fontsize=12)
ax.set_title('モデル比較 - 各フォールドのRMSE分布\n（箱ひげ図：中央線が中央値、箱が四分位範囲）', fontsize=14)
plt.xticks(rotation=15, fontsize=11)
plt.tight_layout()
plt.show()

---
## 9. 提出ファイル作成

### 手順

1. 最良モデルで全訓練データを使って再学習
2. テストデータに対して予測
3. 予測値に `np.expm1` を適用して元のスケールに戻す（`log1p` の逆変換）
4. Kaggle提出形式のCSVファイルを作成

### `expm1` について

`np.expm1(x)` は `np.exp(x) - 1` と同じです。`log1p` の逆変換です。

$$\text{expm1}(\text{log1p}(x)) = e^{\ln(1+x)} - 1 = (1+x) - 1 = x$$

In [ ]:
# ============================================================
# 最良モデルで予測
# ============================================================

print(f'最良モデル: {best_model_name}')
print(f'CV RMSE: {results[best_model_name].mean():.5f}')

# 最良モデルを全訓練データで再学習
best_model = models[best_model_name]
best_model.fit(X_train, y_train)
print(f'\n全訓練データ（{X_train.shape[0]}サンプル）で再学習完了')

# テストデータに対して予測
y_pred_log = best_model.predict(X_test)

# log1pの逆変換（expm1）で元のスケールに戻す
y_pred = np.expm1(y_pred_log)

# 負の予測値がないことを確認（万が一の場合は0にクリップ）
n_negative = (y_pred < 0).sum()
if n_negative > 0:
    print(f'警告: {n_negative}件の負の予測値を0に修正しました')
    y_pred = np.maximum(y_pred, 0)

print(f'\n予測値の統計:')
print(f'  最小値: ${y_pred.min():,.0f}')
print(f'  最大値: ${y_pred.max():,.0f}')
print(f'  平均値: ${y_pred.mean():,.0f}')
print(f'  中央値: ${np.median(y_pred):,.0f}')

In [ ]:
# 予測値の分布を確認
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 訓練データの実際の価格分布
train_prices = np.expm1(y_train)
axes[0].hist(train_prices, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('訓練データの SalePrice 分布', fontsize=13)
axes[0].set_xlabel('SalePrice ($)')
axes[0].set_ylabel('頻度')

# テストデータの予測価格分布
axes[1].hist(y_pred, bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_title('テストデータの予測 SalePrice 分布', fontsize=13)
axes[1].set_xlabel('SalePrice ($)')
axes[1].set_ylabel('頻度')

plt.suptitle('訓練データ vs 予測値の価格分布比較', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print('訓練データと予測値の分布が似ていれば、モデルは適切に学習しています')

In [ ]:
# ============================================================
# 提出ファイルの作成
# ============================================================
import os

# 提出用DataFrameの作成
submission = pd.DataFrame({
    'Id': test_id,
    'SalePrice': y_pred
})

# 提出ディレクトリの確認・作成
submission_dir = '/home/rex/Documents/lb/Kaggle/house-price-prediction/submissions'
os.makedirs(submission_dir, exist_ok=True)

# CSVファイルとして保存
submission_path = os.path.join(submission_dir, 'submission_baseline.csv')
submission.to_csv(submission_path, index=False)

print(f'提出ファイルを保存しました: {submission_path}')
print(f'ファイルサイズ: {os.path.getsize(submission_path) / 1024:.1f} KB')
print(f'\n=== 提出ファイルの先頭10行 ===')
print(submission.head(10).to_string(index=False))

In [ ]:
# サンプル提出ファイルとの形式比較
sample_sub = pd.read_csv('/home/rex/Documents/lb/Kaggle/house-price-prediction/data/raw/sample_submission.csv')

print('=== 形式の確認 ===')
print(f'サンプル提出:  {sample_sub.shape[0]}行, 列={list(sample_sub.columns)}')
print(f'今回の提出:    {submission.shape[0]}行, 列={list(submission.columns)}')

# IDの一致を確認
ids_match = (sample_sub['Id'].values == submission['Id'].values).all()
print(f'\nIDの一致: {"OK" if ids_match else "NG - 修正が必要!"}')
print(f'行数の一致: {"OK" if sample_sub.shape[0] == submission.shape[0] else "NG - 修正が必要!"}')

---
## まとめ

### 実施した前処理

| ステップ | 内容 | 理由 |
|---------|------|------|
| 対数変換 | SalePriceにlog1p変換 | 歪んだ分布を正規化、RMSLE評価指標と整合 |
| 外れ値除去 | GrLivArea > 4000の2件 | 異常な取引がモデルを歪める |
| 欠損値処理 | 意味に応じたNone/0、中央値、最頻値補完 | モデルがNAを扱えないため |
| 特徴量工学 | 総面積、築年数、バイナリフラグ等 | ドメイン知識でモデルを助ける |
| エンコーディング | 順序→ラベル、名義→ワンホット | カテゴリを数値化（順序の有無で方法を変える） |
| スケーリング | StandardScaler | 線形モデルのため特徴量のスケールを統一 |

### 今後の改善案

1. **ハイパーパラメータチューニング**: GridSearchCV や Optuna を使って最適なパラメータを探索
2. **モデルのアンサンブル**: 複数モデルの予測を平均化（Stacking, Blending）
3. **さらなる特徴量エンジニアリング**: 多項式特徴量、交互作用項
4. **特徴量選択**: 重要度の低い特徴量を除去
5. **歪度の修正**: 数値特徴量にもBox-Cox変換を適用

In [ ]:
print('=' * 60)
print('  前処理とベースラインモデル構築 完了！')
print('=' * 60)
print(f'\n  最良モデル: {best_model_name}')
print(f'  CV RMSE:    {results[best_model_name].mean():.5f}')
print(f'  提出ファイル: submission_baseline.csv')
print(f'\n  次のステップ: ハイパーパラメータチューニングとアンサンブル')
print('=' * 60)